In [5]:

from __future__ import print_function
import numpy as np
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from collections import Counter
import time
import random
from random import sample 
import matplotlib.pyplot as plt
import sklearn
from sklearn import tree
from sklearn import preprocessing

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [6]:
df_raw = pd.read_csv('Train.csv', low_memory=False, parse_dates=["saledate"])
df_test = pd.read_csv('Test.csv',low_memory=False, parse_dates=["saledate"])

In [7]:
def thin(dataset,tolerance):
    features = dataset.columns
    
    useful = []
    
    for f in features: 
        
        num_null = dataset[f].isna().sum()
        if(num_null<=tolerance): 
            useful.append(f)
            
    return dataset[useful]

In [8]:
df_new = thin(df_raw,5000)
features = df_new.columns.tolist()
features.remove('SalePrice')
features = features[:-1]


In [9]:
df_t = df_test[features]
Y = df_new['SalePrice']
X = df_new[features]
X_total = pd.concat([X,df_t])

In [10]:
#Feeding some value to the missing data. 
regressive = []
categorical = []
values = {}
for f in features:
    try:
        med = X_total[f].median()
        values[f] = med
        regressive.append(f)
       
    except:
        
        most = X_total[f].value_counts()[X_total[f].value_counts() == X_total[f].value_counts().max()]
        
        values[f] = most
        categorical.append(f)
        
  

X_total = X_total.fillna(value = values)

        
    

In [11]:
le = preprocessing.LabelEncoder()
for f in categorical:
        X_total[f] = X_total[f].astype('category')
        le.fit(X_total[f])
        X_total[f] = le.transform(X_total[f])

In [12]:

X_train = X_total[features][:len(X)-100000]
Y_train = Y[:len(X)-100000]
X_test = X_total[features][len(X)-10000:len(X)]
Y_test = Y[len(X)-10000:len(X)]

In [13]:
m = RandomForestRegressor(n_jobs=-1)
clf = tree.DecisionTreeRegressor()
#Training both Decision Tree and Random Forest. 
m.fit(X_train, Y_train)
clf.fit(X_train,Y_train)

c:\users\abhinav\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [16]:
print("performance using random forest regressor",m.score(X_test,Y_test))
print("performace using decision tree regressor",clf.score(X_test,Y_test))


performance using random forest regressor 0.5983364003886971
performace using decision tree regressor 0.40260147809248054
